In [5]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot
from typing import NamedTuple

In [ ]:
#################### TODO ####################

# write a load function that uses the approach outlined in carpark
# must handle over capacity 

# write a scheduling function that augments the paths of elevators

# We can copy and paste the main code and change scheduling/loading algorithm 

In [3]:
#################### PARAMETERS ####################

# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
# occupancy : max occupancy of an elevator
# h         : length of simulation

#################### SIMULATION DESIGN ####################

# Elevators have predetermined paths
# On button press, scheduler decides which paths change
# When elevator reaches a floor people get off then get on

# Clocks for changes in button states
#        and next stop on elevator path

#################### NOTES ####################

# buttons indicate first arrival of type down/up
# therefore the amount of people that go down/up
# at elevator arrival is that minus time of first arrival

# floors are labeled 0 ... m - 1
# assume time to travel between floors is constant

#################### DATA STRUCTURES ####################

# Feel free to modify, but provide documentation

# down  : tuple where index j is rate of clients of floor j going down
# total : tuple where index j is sum of all rates

# elevators: list of elevator structures

# elevator structure: clients    : list where index j is amount of people want to go to floor j
#                     path       : lists of pairs (floor number, time of arrival to floor, time finish)
#                     floor      : the current floor

# floors: list of floor structures

# floor structure: clients : list where index j is amount of people want to go to floor j
#                  button  : list of pairs (time of press, type of press, floor)

def sim(m: int, n: int, lambdas: tuple[tuple[float]], occupancy: int, h: int):
    class Elevator(NamedTuple):
        clients: list
        path: list
        floor: int
        direction: int

    # Initialization 
    rng       = np.random.default_rng() # rv generator
    t         = 0                       # total elapsed time
    floors    = list( [] for _ in range(m) )
    elevators = list( Elevator(np.zeros(m), [], 0) for _ in range(n) )
    down      = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total     = tuple( sum(lambdas[j]) for j in range(m) )
    clocks    = []

    # Initialize floor/button clocks
    for i in range(m):
        pair = [rng.exponential(1/total[i])] # time of press
        # type of press
        if rng.uniform(0, 1) <= down[i] / total[i]:
            pair.append(-1)
        else:
            pair.append(1)
        pair.append(i)
        clocks.append(pair)
        floors[i].append(pair)
    heapq.heapify(clocks)

    # Finite horizon
    while(t <= h):
        # SCHEDULER SECTION
        time, direction, floor = clocks[0]

        # We need to choose the "best" elevator that will have path modified
        # This is where we can test different policies
        # for now policy will be "closest" elevator
        # the elvator that has a scheduled stop closest to the floor rung
        # ties will be broken by earliest time of arrival to that floor

        # The algorithm below "streams" in the elevators and keeps a running "best" one
        # closest |-> c
        # elevator |-> e
        # index |-> ix
        # time |-> t
        c_distance = floor    # closest distance from a scheduled floor to rung floor
        c_e_arrival_t = -1    # the time of arrival to rung floor by closest elevator
        e_ix = -1             # index of elevator to have path modified
        path_ix = -1          # the index to insert into the path
        for elevator, i in enumerate(elevators):
            if elevator.direction == direction: # Elevator can pick it up 
                # Search through its path to see if its a good candidate
                for e_floor, _, e_finish_t, j in enumerate(elevator.path):
                    if np.abs(e_floor - floor) <= c_distance:
                        c_distance = np.abs(e_floor - floor)
                        if e_ix == -1 or e_finish_t + c_distance <= c_e_arrival_t:
                            c_e_arrival_t = e_finish_t + c_distance # takes one unit of time to move a floor
                            e_ix = i
                            path_ix = j
        # variables now store "closest" path information so we augment the relevant path
        elevators[i].path.insert(j+1, (floor, c_e_arrival_t, 0)) # need to fill 0 with time to finish loading

        #Generate the next button press?



        # ELEVATOR SECTION i.e. calculating loading times, etc. 
        # Will use floor structures so change them to what you like 


    return # performance metrics

SyntaxError: expected ':' (2373290367.py, line 75)

In [4]:
#tracking the clocks of button change of every floor
def DES_engine(m: int, lambda_param, h: int, buckets):
    rng = np.random.default_rng() # rv generator
    t = 0.0
    n = 0
    generate_tuples = lambda m: [(0, 0) for _ in range(m)]
    generate_clocks = lambda m: [rng.exponential(1/lambda_param) for _ in range(m)]
    floor_states = generate_tuples(m) #initial state of every floor is (0,0)
    button_arrival = generate_clocks(m) #generates arrival times for each button on each floor, maybe have to make it a tuple for up and down ?
    q_up = 0
    q_down = 0
    
    SOF = [button_arrival, floor_states, q_up, q_down]
    
    while (t<h):
        next = min(button_arrival)
        tau = next[0]
        floor = button_arrival.index(tau)
        for i in buckets:
            if i < up:
                